<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [83]:
# standard DS stack
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import pandas as pd
# embed static images in the ipynb
%matplotlib inline 

The documentation for this dataset is horrible. I was able to find out that all of the features are numerical, and it says in the dataset description that the target is "number of comments" on a post, so I know it's an integer. 

In [95]:
train_df = pd.read_csv("training.csv", header=None)
test_df = pd.read_csv("testing.csv", header=None)
print("Original dataset shapes\n"
     +f"Training set:{train_df.shape}, Testing set:{test_df.shape}")
train_df.head()

Original dataset shapes
Training set:(40949, 54), Testing set:(10044, 54)


,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,0,1,0
1,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,0,0,1,0,0
2,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,0,0,0,0,0,1,0
3,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,1,0,0,1,0,0,0,0,0,0
4,634995,0,463,1,0.0,806.0,11.291045,1.0,70.495138,0.0,...,0,0,0,0,0,1,0,0,0,0


I think the last column has the targets, so I'll check to see if every value in the last column is an integer.

In [85]:
def integer_check(vec):
    """Args: vec (np.ndarray, 1D): a vector."""
    if np.all(((vec % 1) == 0)) == True:
        print("This vector contains only integers")
    else:
        print("This vector contains non-integer values")

integer_check(vec=np.array(train_df.iloc[:,-1]))

This vector contains only integers


In [86]:
train = np.array(train)
test = np.array(test)
X_train, Y_train = train[:,:-1], train[:,-1] 
X_test, Y_test = test[:,:-1], test[:,-1]

from sklearn.preprocessing import StandardScaler
def scale_features(X_train, X_test):
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

scale_features(X_train, X_test)

# Principal component analysis (PCA) feature reduction
from sklearn.decomposition import PCA
pca = PCA(n_components=10)
X_train = pca.fit_transform(X_train)
X_test = pca.transform(X_test)

# SelectKBest feature selection
# from sklearn.feature_selection import SelectKBest, f_regression
# X_train = SelectKBest(f_regression, k=10).fit_transform(X_train, Y_train)

(40949, 10)

In [88]:
rng = np.random.RandomState(5)
def random_shrink(X, Y, shrink=0.5):
    """Shrinks the dataset size.

    Args:
        X (np.ndarray): feature matrix
        Y (np.ndarray): target matrix
        shrink (float, optional): Percentage of samples desired.
            Defaults to 0.5, i.e. a 50% reduction in the number of samples.

    Returns:
        X_small, Y_small : Random samples of the input sets
    """
    n_samples = X.shape[0] 
    sorted_indices = np.arange(n_samples)
    random_indices = rng.choice(sorted_indices, int(shrink * n_samples))
    X_small = X[random_indices]
    Y_small = Y[random_indices]
    return X_small, Y_small

X_train, Y_train = random_shrink(X_train, Y_train, shrink=0.25)
X_test, Y_test = random_shrink(X_test, Y_test)
print("Dataset shapes after PCA and random sampling\n"
     +f"X_train.shape:{X_train.shape}, Y_train.shape:{Y_train.shape}\n"
     +f"X_test.shape:{X_test.shape}, Y_test.shape:{Y_test.shape}")

((10237, 10), (10237,), (5022, 10), (5022,))

We're not worried about performance, so I'll shrink the training set to 20,000 for time's sake in the tutorial. 